In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [3]:
df = pd.read_csv("audit_5%.csv")

In [4]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,NaN,0.0,0.2,0.0,2.4,8.574,0.4,0.5,NaN,1.0
1,NaN,0.00,NaN,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,NaN,10.80,0.6,6.480,NaN,6.0,0.6,...,0.4,0.0,0.2,NaN,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,NaN,0.2,...,0.4,NaN,NaN,0.0,2.0,1.416,0.4,0.5,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,0.49,0.2,0.098,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578,0.4,NaN,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,0.2,0.074,0.84,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.3136,0.0
773,55.57,0.24,0.2,NaN,0.04,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,0.2,0.040,0.00,0.2,0.000,0.20,NaN,0.2,...,0.4,0.0,0.2,0.0,2.0,1.440,0.4,0.5,0.2880,0.0


In [5]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [6]:
df.isnull().sum()

Sector_score      56
PARA_A            46
Score_A           38
Risk_A            46
PARA_B            43
Score_B           39
Risk_B            46
TOTAL             34
numbers           38
Score_B.1         26
Risk_C            41
Money_Value       43
Score_MV          35
Risk_D            35
District_Loss     24
PROB              45
RiSk_E            48
History           48
Prob              43
Risk_F            40
Score             37
Inherent_Risk     33
CONTROL_RISK      35
Detection_Risk    42
Audit_Risk        49
Risk               0
dtype: int64

In [7]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 1010
persentase : 5.005947660586836


In [8]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score      7.216495
PARA_A            5.927835
Score_A           4.896907
Risk_A            5.927835
PARA_B            5.541237
Score_B           5.025773
Risk_B            5.927835
TOTAL             4.381443
numbers           4.896907
Score_B.1         3.350515
Risk_C            5.283505
Money_Value       5.541237
Score_MV          4.510309
Risk_D            4.510309
District_Loss     3.092784
PROB              5.798969
RiSk_E            6.185567
History           6.185567
Prob              5.541237
Risk_F            5.154639
Score             4.768041
Inherent_Risk     4.252577
CONTROL_RISK      4.510309
Detection_Risk    5.412371
Audit_Risk        6.314433
Risk              0.000000
dtype: float64

# Imputasi

In [65]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
        
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
    max_count = max(counts.values())
    max_vals = [num for num, count in counts.items() if count == max_count]
#     print("max_count:",max_count)
#     print("max_vals:",max_vals)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
            
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,3)
    
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
        
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    df.iloc[index, col] = tertimbang
    print("rata-rata tertimbang : ",tertimbang)
    print("row, col:", index, col)
    print(" ")
    
    
    
        

def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

df2 = df.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)

euc_dist = {}

for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()    
    for row in current:
        jarak(dummy, i)
        Final(dummy, i, 3)

Kolom: 1
[3.89, 55.57, 55.57]
median: 2
min: 3.89
max: 55.57
mean: 38.343333333333334
fuzzy: [(3.89, 55.57), (55.57, 55.57), (55.57, 55.57)]
fuzzy: 55.57
pembilang: 6392.2171
penyebut: 166.71
rata-rata tertimbang: 38.343333333333334
row, col: [1, 16, 30, 35, 45, 56, 82, 84, 85, 88, 109, 133, 187, 190, 193, 200, 206, 218, 225, 249, 273, 283, 288, 309, 322, 335, 369, 370, 392, 398, 418, 443, 462, 477, 487, 492, 498, 523, 534, 539, 556, 593, 599, 610, 614, 631, 656, 671, 676, 684, 693, 698, 732, 754, 770, 771] 0



IndexError: list index out of range

In [25]:
index = dummy.index[dummy.iloc[:,col] == 0].tolist()
index[0]

NameError: name 'col' is not defined

In [ ]:
df2 = df.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)

dummy['numbers'] = dummy['numbers'].fillna(0)

euc_dist = {}

for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,3)

In [ ]:
dummy

In [ ]:
dummy.describe()

# Klasifikasi

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
x = dummy.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}

In [ ]:
classifier = KNeighborsClassifier(metric='euclidean')
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(x_train, y_train)

In [ ]:
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print("Akurasi:", mean_score, "dengan parameter:", params)

In [ ]:
print('Best parameter: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=9, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)